# Chapter 2 -- Intermediate Python
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/CH02_Intermediate_Python.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Part:** 1 -- Core Python Fundamentals  
**Prerequisites:** Chapter 1 (Python Fundamentals)  
**Estimated time:** 5-6 hours

---

### Learning Objectives

By the end of this chapter you will be able to:

- Import and use Python's standard library modules and third-party packages
- Read and write files safely using the `with` statement
- Load and save CSV and JSON data without Pandas
- Handle errors gracefully with `try`/`except`/`else`/`finally`
- Define classes with attributes, methods, and `__dunder__` methods
- Use inheritance to build specialised subclasses
- Recognise how OOP maps directly to scikit-learn and Keras APIs
- Write list, dict, and set comprehensions
- Use generators to process large data without loading it all into memory
- Write and apply decorators
- Use context managers with `with`

---

### Project Thread -- Chapter 2

We build a `SurveyLoader` class that wraps the SO 2025 CSV loading, validation,
and basic cleaning into a reusable object. By the end of this chapter you will have
a working data-loader class -- the same architectural pattern used by PyTorch's
`DataLoader`, scikit-learn's `Pipeline`, and every professional ML codebase.


---

## Section 2.1 -- Modules and Packages

A **module** is any Python file (`.py`). A **package** is a directory of modules.
The `import` statement makes a module's contents available in your current session.

Python ships with an enormous **standard library** -- modules for file handling,
math, dates, networking, compression, and hundreds of other tasks. These are always
available with no installation. Third-party packages (NumPy, Pandas, scikit-learn)
are installed separately with `pip`.

> **Why this matters for AI/ML:** You will write `import` as your first line in
> every notebook for the rest of this book. Understanding what importing actually
> does makes debugging import errors and version conflicts much easier.


In [ ]:
# 2.1.1 -- Import styles
#
# There are four main ways to import:
#   import module              -- access as module.thing
#   import module as alias     -- access as alias.thing (universal convention)
#   from module import name    -- access as name directly
#   from module import *       -- import everything (avoid -- pollutes namespace)

# Style 1: import the whole module
import math
print(f'pi = {math.pi:.6f}')
print(f'sqrt(144) = {math.sqrt(144)}')
print(f'log2(1024) = {math.log2(1024)}')

# Style 2: import with alias -- this IS the convention for data science
import os
import sys
print(f'Python: {sys.version.split()[0]}')
print(f'Working dir: {os.getcwd()}')

# Style 3: import specific names -- useful for commonly-used functions
from datetime import datetime, timedelta
now = datetime.now()
print(f'Current time: {now.strftime("%Y-%m-%d %H:%M")}')
print(f'One week ago: {(now - timedelta(days=7)).strftime("%Y-%m-%d")}')

# Checking what is inside a module with dir()
math_contents = [name for name in dir(math) if not name.startswith('_')]
print(f'Math module exports: {math_contents[:8]}...')


In [ ]:
# 2.1.2 -- Standard library modules you will use in this book

# os -- operating system interface
import os
print('--- os ---')
print(f'  os.getcwd():          {os.getcwd()}')
print(f'  os.path.exists():     {os.path.exists("/tmp")}')
print(f'  os.path.join():       {os.path.join("data", "survey", "file.csv")}')
print(f'  os.path.basename():   {os.path.basename("/data/survey/file.csv")}')

# collections -- specialised container types
from collections import Counter, defaultdict, OrderedDict
print()
print('--- collections ---')
languages = ['Python', 'SQL', 'Python', 'JavaScript', 'Python', 'SQL', 'Rust']
counts = Counter(languages)         # counts occurrences automatically
print(f'  Counter: {dict(counts)}')
print(f'  Most common 3: {counts.most_common(3)}')

# defaultdict: like dict but auto-creates missing keys with a default value
salary_by_country = defaultdict(list)   # missing keys return empty list
data = [('USA', 135_000), ('UK', 88_000), ('USA', 210_000), ('UK', 72_000)]
for country, sal in data:
    salary_by_country[country].append(sal)   # no KeyError if country is new
for country, sals in salary_by_country.items():
    print(f'  {country}: avg ${sum(sals)/len(sals):,.0f}')

# itertools -- tools for efficient looping
import itertools
print()
print('--- itertools ---')
pairs = list(itertools.combinations(['Python', 'SQL', 'JS'], 2))
print(f'  Language pairs: {pairs}')

# random -- random number generation (we used this in Chapter 1)
import random
random.seed(42)
sample_idx = random.sample(range(15_000), 5)   # 5 random row indices from 15k
print(f'  5 random indices: {sample_idx}')


---

## Section 2.2 -- File I/O: Reading and Writing Files

File I/O (Input/Output) is how Python reads from and writes to files on disk.
In data science you will constantly read CSVs, write processed outputs, save
model results, and load configuration files.

Python's `with` statement is the safe way to open files -- it guarantees the file
is closed properly even if an error occurs mid-read. **Never use `open()` without
`with` in production code.**


In [ ]:
# 2.2.1 -- Reading and writing plain text files

import os

# Write a small text file -- 'w' mode creates or overwrites
file_path = '/tmp/survey_notes.txt'

# The 'with' statement opens the file, gives us the handle 'f',
# and automatically closes it when the block exits -- even on error.
with open(file_path, 'w', encoding='utf-8') as f:
    f.write('SO 2025 Developer Survey -- Analysis Notes\n')
    f.write('=' * 45 + '\n')
    f.write('Dataset: 15,000 respondents (curated subset)\n')
    f.write('Primary target: ConvertedCompYearly (salary USD)\n')

print(f'Written: {file_path}  ({os.path.getsize(file_path)} bytes)')

# Read it back -- 'r' mode (default) opens for reading
with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()   # read entire file as one string
print('File contents:')
print(content)

# Append to an existing file -- 'a' mode adds to the end
with open(file_path, 'a', encoding='utf-8') as f:
    f.write('Added: additional note on second open\n')

# Read line by line -- efficient for large files (one line in memory at a time)
print('Line by line:')
with open(file_path, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, start=1):
        print(f'  Line {line_num}: {line.rstrip()}')   # .rstrip() removes trailing newline


In [ ]:
# 2.2.2 -- CSV files with the built-in csv module
#
# CSV (Comma-Separated Values) is the most common format for tabular data.
# Python's csv module handles quoting, escaping, and dialects correctly.
# (In Chapter 3 we switch to Pandas -- but knowing the raw approach matters.)

import csv

csv_path = '/tmp/sample_respondents.csv'

# Writing CSV
respondents = [
    {'id': 'R001', 'country': 'USA',     'salary': 135000, 'language': 'Python'},
    {'id': 'R002', 'country': 'UK',      'salary':  88000, 'language': 'JavaScript'},
    {'id': 'R003', 'country': 'Germany', 'salary': 105000, 'language': 'Python'},
    {'id': 'R004', 'country': 'India',   'salary':  42000, 'language': 'Java'},
    {'id': 'R005', 'country': 'Canada',  'salary': 118000, 'language': 'Python'},
]

fieldnames = ['id', 'country', 'salary', 'language']

with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    # DictWriter writes dicts as rows, using fieldnames as the header
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()       # writes the column name row
    writer.writerows(respondents)   # writes all data rows

print(f'CSV written: {csv_path}')

# Reading CSV
with open(csv_path, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)   # each row becomes a dict keyed by column name
    loaded = list(reader)        # materialise into a list of dicts

print(f'Rows loaded: {len(loaded)}')
for row in loaded:
    print(f'  {row["id"]}  {row["country"]:<10}  ${int(row["salary"]):,}  {row["language"]}')


In [ ]:
# 2.2.3 -- JSON files
#
# JSON (JavaScript Object Notation) is the standard for API responses,
# configuration files, and nested data structures.
# Python's json module maps JSON directly to Python dicts and lists.

import json

json_path = '/tmp/survey_config.json'

# Any Python object that is JSON-serialisable can be written
config = {
    'dataset': 'so_survey_2025_curated.csv',
    'target_column': 'ConvertedCompYearly',
    'feature_columns': ['YearsCodePro', 'Country', 'EdLevel', 'RemoteWork'],
    'salary_filter': {'min': 10_000, 'max': 500_000},
    'random_seed': 42,
    'test_size': 0.2,
}

# json.dump() writes to a file  (json.dumps() returns a string)
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2)   # indent=2 makes it human-readable

print(f'JSON written: {json_path}')

# json.load() reads from a file  (json.loads() parses a string)
with open(json_path, 'r', encoding='utf-8') as f:
    loaded_config = json.load(f)

print('Loaded config:')
for key, value in loaded_config.items():
    print(f'  {key:<20}: {value}')

# Round-trip verification
assert loaded_config['random_seed'] == 42
assert loaded_config['salary_filter']['min'] == 10_000
print('Round-trip verification passed.')


### 2.2.4 -- Project: Loading the SO 2025 CSV with Error Handling

We load the SO 2025 dataset from GitHub using `urllib` and `csv` -- the same approach
as Chapter 1, but now wrapped in proper error handling so the loader fails gracefully
rather than crashing on network issues, malformed rows, or missing columns.


In [ ]:
# 2.2.4 -- Robust CSV loader with error handling

import csv
import urllib.request

DATASET_URL = 'https://raw.githubusercontent.com/timothy-watt/python-for-ai-ml/main/data/so_survey_2025_curated.csv'

def load_so_survey(url, max_rows=50, required_cols=None):
    """
    Load SO 2025 survey data from a URL into a list of dicts.

    Parameters
    ----------
    url : str
        Raw CSV URL to fetch.
    max_rows : int
        Maximum number of valid rows to return.
    required_cols : list of str, optional
        Columns that must be non-empty for a row to be kept.

    Returns
    -------
    list of dict, or None if the fetch failed.
    """
    if required_cols is None:
        required_cols = ['ConvertedCompYearly', 'Country']

    try:
        with urllib.request.urlopen(url, timeout=15) as response:
            raw = response.read().decode('utf-8')
    except urllib.error.URLError as e:
        print(f'Network error: {e.reason}')
        return None
    except Exception as e:
        print(f'Unexpected fetch error: {e}')
        return None

    rows = []
    skipped = 0
    reader = csv.DictReader(raw.splitlines())

    for row in reader:
        # Skip rows missing any required column
        if any(not row.get(col, '').strip() for col in required_cols):
            skipped += 1
            continue

        # Convert salary to float safely
        try:
            row['ConvertedCompYearly'] = float(row['ConvertedCompYearly'])
        except (ValueError, KeyError):
            skipped += 1
            continue

        rows.append(dict(row))
        if len(rows) >= max_rows:
            break

    print(f'Loaded {len(rows)} rows  (skipped {skipped} incomplete rows)')
    return rows


survey_data = load_so_survey(DATASET_URL, max_rows=50)

if survey_data:
    salaries = [r['ConvertedCompYearly'] for r in survey_data]
    countries = set(r['Country'] for r in survey_data)
    print(f'Countries represented: {len(countries)}')
    print(f'Salary range: ${min(salaries):,.0f} to ${max(salaries):,.0f}')
    print(f'Mean salary:  ${sum(salaries)/len(salaries):,.0f}')


---

## Section 2.3 -- Error and Exception Handling

Errors are inevitable in data science code -- missing files, malformed data,
type mismatches, network timeouts. Python's exception handling lets you respond
to errors gracefully rather than crashing.

The full syntax is:
```python
try:
    # code that might raise an exception
except SomeError as e:
    # runs if SomeError (or a subclass) was raised
except AnotherError:
    # multiple except blocks handle different error types
else:
    # runs only if NO exception was raised in the try block
finally:
    # ALWAYS runs -- use for cleanup (closing files, releasing resources)
```


In [ ]:
# 2.3.1 -- Common exception types in data science code

print('--- ValueError: wrong type of value ---')
try:
    salary = float('not_a_number')   # csv column that should be numeric
except ValueError as e:
    print(f'  Caught ValueError: {e}')

print()
print('--- KeyError: missing dict key ---')
row = {'Country': 'USA', 'YearsCodePro': '7'}
try:
    salary = row['ConvertedCompYearly']   # column absent from this row
except KeyError as e:
    print(f'  Caught KeyError: {e}')
    salary = 0.0   # use a default instead of crashing

print()
print('--- IndexError: list index out of range ---')
results = [0.91, 0.93, 0.89]
try:
    fifth_result = results[10]   # only 3 items
except IndexError as e:
    print(f'  Caught IndexError: {e}')

print()
print('--- TypeError: wrong type for operation ---')
try:
    total = '135000' + 88000   # string + int not valid
except TypeError as e:
    print(f'  Caught TypeError: {e}')
    total = int('135000') + 88000   # fix by converting first
    print(f'  Fixed: {total:,}')

print()
print('--- ZeroDivisionError ---')
try:
    avg = sum([]) / 0   # empty list -- dividing by zero
except ZeroDivisionError:
    print('  Caught ZeroDivisionError: no data to average')
    avg = 0.0


In [ ]:
# 2.3.2 -- try / except / else / finally in full

def safe_average(values, label=''):
    """
    Compute the mean of a list, handling all common failure cases.
    Demonstrates the full try/except/else/finally pattern.
    """
    try:
        # This block runs first -- the 'risky' code
        numeric = [float(v) for v in values if str(v).strip()]   # convert all to float
        result  = sum(numeric) / len(numeric)   # ZeroDivisionError if numeric is empty

    except ValueError as e:
        # Runs if any value could not be converted to float
        print(f'  [{label}] Value error: {e} -- returning None')
        return None

    except ZeroDivisionError:
        # Runs if the list was empty after filtering
        print(f'  [{label}] No valid values -- returning None')
        return None

    except Exception as e:
        # Catch-all for anything unexpected -- log it, do not swallow silently
        print(f'  [{label}] Unexpected error: {type(e).__name__}: {e}')
        return None

    else:
        # Runs ONLY if the try block succeeded with no exceptions
        print(f'  [{label}] Success: mean = {result:,.2f} (n={len(numeric)})')

    finally:
        # ALWAYS runs -- even if an exception was raised and caught
        # Use for cleanup: closing files, releasing locks, logging
        pass   # nothing to clean up here

    return result


# Test all paths
safe_average([95_000, 135_000, 88_000, 210_000], label='valid data')
safe_average(['95000', 'not_a_number', '88000'],  label='bad value')
safe_average([],                                   label='empty list')
safe_average([None, '', '  '],                     label='all empty strings')


In [ ]:
# 2.3.3 -- Raising your own exceptions
#
# Sometimes YOU want to signal that something has gone wrong.
# Raise built-in exceptions for standard situations;
# define custom exceptions for domain-specific errors.

class SalaryValidationError(ValueError):
    """Raised when a salary value fails business validation rules."""
    pass   # inherits everything from ValueError; we just gave it a new name


def validate_salary(salary, label=''):
    """Validate that a salary is plausible. Raises SalaryValidationError if not."""
    if not isinstance(salary, (int, float)):
        raise TypeError(f'Salary must be numeric, got {type(salary).__name__}')
    if salary < 0:
        raise SalaryValidationError(f'Negative salary not valid: {salary}')
    if salary < 1_000:
        raise SalaryValidationError(f'Salary suspiciously low (< $1k): {salary}')
    if salary > 10_000_000:
        raise SalaryValidationError(f'Salary suspiciously high (> $10M): {salary}')
    return salary


test_cases = [135_000, -5_000, 500, 15_000_000, 'eighty thousand']

for val in test_cases:
    try:
        result = validate_salary(val)
        print(f'  ${result:>12,.0f}  -- valid')
    except SalaryValidationError as e:
        print(f'  {str(val):>14}  -- SalaryValidationError: {e}')
    except TypeError as e:
        print(f'  {str(val):>14}  -- TypeError: {e}')


---

## Section 2.4 -- Object-Oriented Programming

Object-Oriented Programming (OOP) organises code around **objects** -- bundles
of data (attributes) and behaviour (methods). A **class** is the blueprint;
an **instance** is one concrete object built from that blueprint.

OOP is the reason scikit-learn and Keras look the way they do:

```python
scaler = StandardScaler()   # create an instance
scaler.fit(X_train)         # call a method that stores learned parameters
X_scaled = scaler.transform(X_test)  # call a method that uses those parameters
```

After this section you will understand exactly why that pattern works -- and
be able to build your own classes that follow the same design.


In [ ]:
# 2.4.1 -- Defining a class with __init__ and methods

class SurveyRespondent:
    """
    Represents one SO 2025 Developer Survey respondent.

    Attributes
    ----------
    respondent_id : str
    country : str
    salary : float  -- annual compensation in USD
    languages : list of str
    years_exp : int or None
    """

    # Class variable: shared by ALL instances
    # Changed here, it changes for every SurveyRespondent object.
    survey_year = 2025

    def __init__(self, respondent_id, country, salary, languages=None, years_exp=None):
        """
        Initialiser -- called automatically when you write SurveyRespondent(...).
        'self' refers to the specific instance being created.
        """
        # Instance variables: unique to each object
        self.respondent_id = respondent_id
        self.country       = country
        self.salary        = float(salary)
        self.languages     = languages or []   # default to empty list if None passed
        self.years_exp     = years_exp

    def salary_band(self):
        """Return a salary band label based on annual compensation."""
        if self.salary >= 200_000: return 'Principal'
        if self.salary >= 150_000: return 'Senior+'
        if self.salary >= 100_000: return 'Senior'
        if self.salary >=  60_000: return 'Mid-level'
        return 'Junior'

    def uses_python(self):
        """Return True if Python is in this respondent's language list."""
        return 'Python' in self.languages

    def summary(self):
        """Return a formatted one-line summary string."""
        langs = ', '.join(self.languages[:3])   # show up to 3 languages
        if len(self.languages) > 3:
            langs += f' (+{len(self.languages)-3} more)'
        return (f'[{self.respondent_id}] {self.country} | '
                f'${self.salary:,.0f} ({self.salary_band()}) | '
                f'{langs}')

    # __repr__ is called when you print the object or inspect it in a REPL
    def __repr__(self):
        return f'SurveyRespondent(id={self.respondent_id!r}, country={self.country!r}, salary={self.salary:.0f})'

    # __len__ lets len(respondent) work -- returns number of languages
    def __len__(self):
        return len(self.languages)


# Create instances
r1 = SurveyRespondent('R001', 'USA',     135_000, ['Python', 'SQL', 'JavaScript'], 7)
r2 = SurveyRespondent('R002', 'UK',       88_000, ['JavaScript', 'TypeScript'],    3)
r3 = SurveyRespondent('R003', 'Germany', 105_000, ['Python', 'SQL', 'Rust', 'Go'], 9)

print(r1)                      # calls __repr__
print(r1.summary())
print(r2.summary())
print(r3.summary())
print()
print(f'r1 uses Python:   {r1.uses_python()}')
print(f'r2 uses Python:   {r2.uses_python()}')
print(f'len(r1) [langs]:  {len(r1)}')
print(f'Survey year:      {SurveyRespondent.survey_year}')   # class variable


In [ ]:
# 2.4.2 -- Dunder (magic) methods
#
# Dunder methods (double-underscore on both sides) let your class integrate
# with Python's built-in syntax and functions:
#   __str__      called by str() and print()
#   __repr__     called in REPL and for debugging
#   __eq__       called by ==
#   __lt__       called by < (and sorted())
#   __len__      called by len()
#   __contains__ called by 'in'

class SalaryRecord:
    """A comparable salary record -- demonstrates sorting and equality via dunders."""

    def __init__(self, respondent_id, salary):
        self.respondent_id = respondent_id
        self.salary = salary

    def __repr__(self):
        return f'SalaryRecord({self.respondent_id!r}, ${self.salary:,.0f})'

    def __str__(self):
        return f'{self.respondent_id}: ${self.salary:,.0f}'

    def __eq__(self, other):
        # == compares salary values, not object identity
        if not isinstance(other, SalaryRecord):
            return NotImplemented
        return self.salary == other.salary

    def __lt__(self, other):
        # < enables sorted() to work on a list of SalaryRecords
        return self.salary < other.salary

    def __add__(self, other):
        # + operator -- returns a new SalaryRecord with combined salary (for demo)
        return SalaryRecord(f'{self.respondent_id}+{other.respondent_id}',
                            self.salary + other.salary)


records = [
    SalaryRecord('R001', 135_000),
    SalaryRecord('R002',  88_000),
    SalaryRecord('R003', 210_000),
    SalaryRecord('R004',  72_000),
]

print('Unsorted:', records)
print('Sorted:  ', sorted(records))         # uses __lt__
print('Max:     ', max(records))             # uses __lt__
print('Equal?   ', records[0] == SalaryRecord('X', 135_000))  # uses __eq__
combined = records[0] + records[1]           # uses __add__
print(f'Combined: {combined}')               # uses __str__


In [ ]:
# 2.4.3 -- Inheritance
#
# Inheritance lets a 'child' class reuse and extend a 'parent' class.
# The child gets all the parent's methods automatically, and can:
#   - Add new methods
#   - Override existing methods with new behaviour
#   - Call the parent's version with super()

class SeniorRespondent(SurveyRespondent):
    """
    A SurveyRespondent who is senior (>= 5 years experience).
    Inherits everything from SurveyRespondent and adds mentorship tracking.
    """

    def __init__(self, respondent_id, country, salary, languages=None,
                 years_exp=None, mentors=0):
        # super().__init__() calls the parent class's __init__
        # so we do not duplicate that code here
        super().__init__(respondent_id, country, salary, languages, years_exp)
        self.mentors = mentors   # new attribute specific to SeniorRespondent

    def summary(self):
        # Override the parent's summary() method with a more detailed version
        base = super().summary()   # get the parent's summary string
        return f'{base} | mentors={self.mentors}'

    def is_eligible_principal(self):
        """Senior respondent is eligible for Principal if exp >= 10 and salary >= 180k."""
        exp = self.years_exp or 0
        return exp >= 10 and self.salary >= 180_000


senior = SeniorRespondent('R010', 'USA', 195_000,
                          languages=['Python', 'Scala', 'SQL'],
                          years_exp=12, mentors=3)

print(senior.summary())                         # uses overridden summary()
print(f'Uses Python:         {senior.uses_python()}')    # inherited from parent
print(f'Salary band:         {senior.salary_band()}')    # inherited from parent
print(f'Principal eligible:  {senior.is_eligible_principal()}')  # new method
print()
# isinstance() checks the full inheritance chain
print(f'Is SeniorRespondent:  {isinstance(senior, SeniorRespondent)}')
print(f'Is SurveyRespondent:  {isinstance(senior, SurveyRespondent)}')


### 2.4.4 -- Bridge: How OOP Maps to scikit-learn and Keras

This is one of the most important conceptual bridges in the book.
Every scikit-learn transformer and estimator follows *exactly* the pattern
you just learned. Once you see this, the entire scikit-learn and Keras APIs
become predictable and easy to read.


In [ ]:
# 2.4.4 -- The fit/transform pattern: OOP in disguise
#
# scikit-learn's API is built entirely on two ideas from OOP:
#   1. __init__ sets configuration (hyperparameters)
#   2. fit() learns from data and stores results as instance attributes
#   3. transform() / predict() uses those stored results
#
# Here we build a MinMaxScaler from scratch using pure Python OOP.
# Then we show the identical interface from scikit-learn.

class MinMaxScaler:
    """
    Scales values to [0, 1] by subtracting the min and dividing by the range.
    Mirrors scikit-learn's MinMaxScaler API exactly.
    """

    def __init__(self, feature_range=(0, 1)):
        # __init__ stores CONFIGURATION -- not yet learned from data
        self.feature_range = feature_range
        self.min_   = None   # will be set by fit()
        self.scale_ = None   # will be set by fit()

    def fit(self, values):
        """
        Learn min and scale from the training data.
        Stores them as instance attributes (note the trailing underscore --
        scikit-learn convention: attributes set by fit() end in _).
        """
        self.min_   = min(values)
        data_range  = max(values) - min(values)
        self.scale_ = data_range if data_range != 0 else 1.0
        return self   # return self so fit() can be chained: scaler.fit(X).transform(X)

    def transform(self, values):
        """
        Apply the learned scaling.
        Requires fit() to have been called first.
        """
        if self.min_ is None:
            raise RuntimeError('Call fit() before transform()')
        lo, hi = self.feature_range
        return [lo + (v - self.min_) / self.scale_ * (hi - lo) for v in values]

    def fit_transform(self, values):
        """Convenience: fit and transform in one call."""
        return self.fit(values).transform(values)


# Use our hand-built scaler
salaries = [72_000, 88_000, 105_000, 135_000, 155_000, 195_000, 210_000]

scaler = MinMaxScaler()          # create instance -- no data yet
scaler.fit(salaries)             # learn min and scale from training data
scaled = scaler.transform(salaries)  # apply the scaling

print('Our MinMaxScaler:')
for sal, sc in zip(salaries, scaled):
    print(f'  ${sal:>10,}  ->  {sc:.4f}')
print(f'  Learned min_:   {scaler.min_:,}')
print(f'  Learned scale_: {scaler.scale_:,}')

print()

# Now the IDENTICAL API from scikit-learn
from sklearn.preprocessing import MinMaxScaler as SklearnScaler
import numpy as np

sk_scaler = SklearnScaler()
X = np.array(salaries).reshape(-1, 1)   # sklearn needs a 2D array
sk_scaler.fit(X)
sk_scaled = sk_scaler.transform(X).flatten()

print('scikit-learn MinMaxScaler (same API, same results):')
for sal, sc in zip(salaries, sk_scaled):
    print(f'  ${sal:>10,}  ->  {sc:.4f}')
print()
print('The OOP pattern is identical. This is why understanding classes')
print('makes every scikit-learn and Keras API immediately readable.')


---

## Section 2.5 -- Advanced Techniques: Comprehensions, Generators, Decorators

These three features are what make Python code feel professional.
You will see them constantly in library source code, production ML pipelines,
and code reviews. Knowing them makes you a confident Python reader and writer.


In [ ]:
# 2.5.1 -- List, Dict, and Set Comprehensions
#
# A comprehension is a compact, readable way to build a list, dict, or set
# from an existing iterable -- without writing an explicit for loop.
#
# Syntax:
#   [expression  for item in iterable  if condition]
#   {key: value  for item in iterable  if condition}
#   {expression  for item in iterable  if condition}

salaries  = [72_000, 88_000, 0, 105_000, -1, 135_000, 155_000, 195_000]
countries = ['USA', 'UK', 'Germany', 'India', 'Canada', 'France']

# List comprehension
valid_salaries = [s for s in salaries if s > 0]   # filter out invalid values
print(f'Valid salaries:   {valid_salaries}')

scaled_salaries = [s / 200_000 for s in valid_salaries]   # transform
print(f'Scaled (0-1 ish): {[round(x, 3) for x in scaled_salaries]}')

# Dict comprehension
salary_map = {f'R{i+1:03d}': s for i, s in enumerate(valid_salaries)}
print(f'Salary map: {salary_map}')

# Set comprehension -- unique first letters of countries
first_letters = {c[0] for c in countries}
print(f'First letters: {sorted(first_letters)}')

# Nested comprehension -- all (country, salary) pairs above $100k
high_pairs = [(c, s) for c in countries[:3] for s in valid_salaries if s > 100_000]
print(f'High-salary country pairs: {high_pairs[:4]}...')

# Equivalent for-loop vs comprehension -- same result, comprehension is faster
loop_result = []
for s in salaries:
    if s > 0:
        loop_result.append(s)
assert loop_result == valid_salaries, 'Results must match'
print('Loop and comprehension produce identical results: confirmed')


In [ ]:
# 2.5.2 -- Generators: Memory-Efficient Iteration
#
# A generator produces values ONE AT A TIME on demand -- it never
# builds the full sequence in memory. This matters enormously when
# processing large datasets that do not fit in RAM.
#
# A generator function uses 'yield' instead of 'return'.
# A generator expression is a comprehension with () instead of [].

import sys

# Compare memory usage: list vs generator
big_list = [s * 1.1 for s in range(100_000)]        # builds full list in memory
big_gen  = (s * 1.1 for s in range(100_000))         # generates on demand

print(f'List size:      {sys.getsizeof(big_list):>10,} bytes')
print(f'Generator size: {sys.getsizeof(big_gen):>10,} bytes')
print(f'Memory ratio:   {sys.getsizeof(big_list) / sys.getsizeof(big_gen):.0f}x more for the list')

# Generator function -- streams rows from CSV one at a time
def stream_valid_salaries(rows, min_salary=10_000, max_salary=500_000):
    """
    Generator that yields valid salary values one at a time.
    Handles an arbitrarily large list without loading all values.
    """
    for row in rows:
        try:
            sal = float(row.get('salary', 0) or 0)
        except (ValueError, TypeError):
            continue   # skip malformed values
        if min_salary <= sal <= max_salary:
            yield sal   # 'yield' pauses the function and sends the value
                        # execution resumes here on the next next() call

# Simulate a dataset
fake_rows = [
    {'salary': 135_000}, {'salary': 0}, {'salary': 88_000},
    {'salary': None},    {'salary': 210_000}, {'salary': 'bad'},
    {'salary': 72_000},  {'salary': 600_000},  # too high -- filtered
]

salary_gen = stream_valid_salaries(fake_rows)
print()
print('Streaming valid salaries:')
total = 0
count = 0
for sal in salary_gen:   # iterate the generator -- one value at a time
    print(f'  ${sal:,.0f}')
    total += sal
    count += 1

print(f'Mean: ${total/count:,.0f}  (n={count})')


In [ ]:
# 2.5.3 -- Decorators
#
# A decorator is a function that WRAPS another function to add behaviour
# before or after it runs -- without modifying the original function.
# Syntax: @decorator_name above the function definition.
#
# You have already used decorators without knowing it:
#   @property, @staticmethod, @classmethod in OOP
#   @app.route() in Flask/FastAPI web frameworks

import time
import functools

# Decorator 1: timer -- measures how long a function takes
def timer(func):
    """
    Decorator that prints the execution time of any function.
    functools.wraps preserves the original function's name and docstring.
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start  = time.perf_counter()         # high-resolution timer
        result = func(*args, **kwargs)       # call the original function
        end    = time.perf_counter()
        print(f'  [{func.__name__}] took {(end-start)*1000:.2f} ms')
        return result
    return wrapper


# Decorator 2: validate_input -- checks that salary is positive
def require_positive_salary(func):
    """Decorator that raises ValueError if the first argument is not positive."""
    @functools.wraps(func)
    def wrapper(salary, *args, **kwargs):
        if salary <= 0:
            raise ValueError(f'{func.__name__}: salary must be positive, got {salary}')
        return func(salary, *args, **kwargs)
    return wrapper


# Apply decorators with @ syntax
@timer
@require_positive_salary
def compute_tax(salary, rate=0.30):
    """Compute tax owed at a given rate."""
    time.sleep(0.001)   # simulate a small computation
    return salary * rate


print('Decorated function calls:')
tax = compute_tax(135_000)
print(f'  Tax on $135,000 @ 30%: ${tax:,.0f}')

tax = compute_tax(88_000, rate=0.25)
print(f'  Tax on $88,000 @ 25%:  ${tax:,.0f}')

print()
try:
    compute_tax(-5_000)
except ValueError as e:
    print(f'  Caught: {e}')


In [ ]:
# 2.5.4 -- Context Managers with 'with'
#
# You have already used context managers: 'with open(...)' is the most common.
# Here we see HOW they work and how to write your own.
#
# A context manager implements __enter__ and __exit__ methods.
# __enter__ runs at the start of the 'with' block.
# __exit__ runs at the end -- even if an exception occurred.

import time

class Timer:
    """Context manager that times a block of code."""

    def __init__(self, label=''):
        self.label = label

    def __enter__(self):
        self.start = time.perf_counter()
        return self   # the value bound to 'as t' in 'with Timer() as t'

    def __exit__(self, exc_type, exc_val, exc_tb):
        # exc_type is None if no exception occurred
        elapsed = (time.perf_counter() - self.start) * 1000
        status  = 'ERROR' if exc_type else 'OK'
        print(f'  [{self.label}] {elapsed:.2f} ms  ({status})')
        return False   # False = do not suppress exceptions


print('Context manager timing blocks:')

with Timer('list comprehension') as t:
    result = [x ** 2 for x in range(50_000)]

with Timer('generator sum') as t:
    result = sum(x ** 2 for x in range(50_000))

# contextlib.contextmanager: write context managers as generators
from contextlib import contextmanager

@contextmanager
def log_step(name):
    """Log the start and end of a processing step."""
    print(f'  Starting: {name}')
    try:
        yield   # the body of the 'with' block runs here
    finally:
        print(f'  Finished: {name}')

print()
with log_step('data cleaning'):
    time.sleep(0.01)   # simulated work

with log_step('feature engineering'):
    time.sleep(0.01)


---

## Section 2.5.5 -- Project: SurveyLoader Class

We now build the `SurveyLoader` class that ties together everything from Chapter 2:
modules, file I/O, error handling, OOP, and the fit/transform pattern.

`SurveyLoader` follows the same architectural pattern as scikit-learn's
`Pipeline` and PyTorch's `DataLoader` -- a class that encapsulates all
the complexity of data loading behind a clean, predictable interface.


In [ ]:
import csv
import json
import urllib.request
import urllib.error
from collections import Counter

DATASET_URL = 'https://raw.githubusercontent.com/timothy-watt/python-for-ai-ml/main/data/so_survey_2025_curated.csv'

class SurveyLoader:
    """
    Loads, validates, and provides access to SO 2025 Developer Survey data.

    Follows the fit/load pattern:
        loader = SurveyLoader(url)
        loader.load()               # fetch and parse the CSV
        loader.validate()           # check data quality
        df = loader.get_records()   # return the cleaned data

    Attributes set after load():
        records_      : list of dicts -- all loaded rows
        column_names_ : list of str -- column headers
        n_rows_       : int -- number of loaded rows
    """

    # Columns we require to be non-empty for a row to be kept
    REQUIRED_COLS = ['ConvertedCompYearly', 'Country']

    # Salary bounds for outlier filtering
    SALARY_MIN =      5_000
    SALARY_MAX = 10_000_000

    def __init__(self, url, max_rows=None, random_seed=42):
        self.url         = url
        self.max_rows    = max_rows
        self.random_seed = random_seed
        # Attributes set by load() -- trailing _ follows scikit-learn convention
        self.records_      = None
        self.column_names_ = None
        self.n_rows_       = 0
        self._is_loaded    = False

    def load(self):
        """
        Fetch the CSV from the URL and parse into a list of dicts.
        Sets self.records_, self.column_names_, self.n_rows_.
        Returns self for method chaining.
        """
        print(f'Loading: {self.url}')
        try:
            with urllib.request.urlopen(self.url, timeout=20) as resp:
                raw = resp.read().decode('utf-8')
        except urllib.error.URLError as e:
            raise ConnectionError(f'Could not reach dataset URL: {e.reason}') from e

        reader = csv.DictReader(raw.splitlines())
        self.column_names_ = reader.fieldnames or []

        records = []
        skipped = 0
        for row in reader:
            # Filter: required columns must be non-empty
            if any(not row.get(c, '').strip() for c in self.REQUIRED_COLS):
                skipped += 1
                continue
            # Filter: salary must be a valid number in range
            try:
                sal = float(row['ConvertedCompYearly'])
                if not (self.SALARY_MIN <= sal <= self.SALARY_MAX):
                    skipped += 1
                    continue
                row['ConvertedCompYearly'] = sal   # store as float
            except (ValueError, KeyError):
                skipped += 1
                continue

            records.append(dict(row))
            if self.max_rows and len(records) >= self.max_rows:
                break

        self.records_   = records
        self.n_rows_    = len(records)
        self._is_loaded = True
        print(f'  Loaded {self.n_rows_:,} rows  (skipped {skipped:,} incomplete/invalid)')
        return self

    def validate(self):
        """Run data quality checks. Prints a report. Returns self."""
        self._require_loaded()
        print('Data Quality Report:')
        print(f'  Total rows:       {self.n_rows_:,}')
        print(f'  Total columns:    {len(self.column_names_)}')
        # Missing value counts per column
        missing = {}
        for col in self.column_names_:
            n_missing = sum(1 for r in self.records_ if not str(r.get(col, '')).strip())
            if n_missing > 0:
                missing[col] = n_missing
        if missing:
            print(f'  Columns with missing values:')
            for col, n in sorted(missing.items(), key=lambda x: -x[1])[:5]:
                pct = n / self.n_rows_ * 100
                print(f'    {col:<35} {n:>5,} ({pct:.1f}%)')
        else:
            print('  No missing values in required columns.')
        return self

    def get_records(self, columns=None):
        """
        Return the loaded records, optionally selecting specific columns.
        Returns a list of dicts.
        """
        self._require_loaded()
        if columns is None:
            return self.records_
        return [{c: r.get(c) for c in columns} for r in self.records_]

    def salary_summary(self):
        """Print salary statistics."""
        self._require_loaded()
        sals = sorted(r['ConvertedCompYearly'] for r in self.records_)
        n    = len(sals)
        mean = sum(sals) / n
        mid  = n // 2
        median = sals[mid] if n % 2 else (sals[mid-1] + sals[mid]) / 2
        print(f'Salary Summary (n={n:,}):')
        print(f'  Mean:   ${mean:>10,.0f}')
        print(f'  Median: ${median:>10,.0f}')
        print(f'  Min:    ${sals[0]:>10,.0f}')
        print(f'  Max:    ${sals[-1]:>10,.0f}')

    def top_countries(self, n=5):
        """Print the top n countries by respondent count."""
        self._require_loaded()
        counts = Counter(r['Country'] for r in self.records_)
        print(f'Top {n} countries:')
        for country, count in counts.most_common(n):
            print(f'  {country:<30} {count:>5,} respondents')

    def _require_loaded(self):
        """Internal helper: raise if load() has not been called yet."""
        if not self._is_loaded:
            raise RuntimeError('Call load() before using this method.')

    def __repr__(self):
        status = f'{self.n_rows_:,} rows loaded' if self._is_loaded else 'not yet loaded'
        return f'SurveyLoader(url={self.url!r}, status={status!r})'


# Run the full loader pipeline
loader = SurveyLoader(DATASET_URL, max_rows=200)
loader.load().validate()   # method chaining: load() returns self
print()
loader.salary_summary()
print()
loader.top_countries(n=8)
print()
print(repr(loader))


In [ ]:
# 2.5.6 -- Extending SurveyLoader: a subclass with language analytics
#
# Demonstrates inheritance from our own class -- same pattern as
# subclassing PyTorch's nn.Module or scikit-learn's BaseEstimator.

class SurveyAnalyser(SurveyLoader):
    """
    Extends SurveyLoader with language and AI tool analytics.
    Inherits all loading and validation from SurveyLoader.
    """

    def top_languages(self, n=10):
        """
        Count language popularity across all respondents.
        Languages are semicolon-separated; we split and flatten.
        """
        self._require_loaded()
        all_langs = []
        for r in self.records_:
            lang_str = r.get('LanguageHaveWorkedWith', '')
            if lang_str:
                all_langs.extend(l.strip() for l in lang_str.split(';') if l.strip())
        counts = Counter(all_langs)
        print(f'Top {n} languages (from {len(self.records_):,} respondents):')
        for rank, (lang, count) in enumerate(counts.most_common(n), 1):
            pct = count / len(self.records_) * 100
            bar = '#' * int(pct / 2)
            print(f'  #{rank:>2}: {lang:<25} {count:>5,} ({pct:>5.1f}%)  {bar}')

    def ai_tool_adoption(self):
        """Summarise AI tool usage."""
        self._require_loaded()
        using_ai   = sum(1 for r in self.records_
                         if r.get('AIToolCurrently', '').strip())
        pct        = using_ai / len(self.records_) * 100
        print(f'AI Tool Adoption:')
        print(f'  Respondents using AI tools: {using_ai:,} of {len(self.records_):,} ({pct:.1f}%)')
        all_tools = []
        for r in self.records_:
            tools_str = r.get('AIToolCurrently', '')
            if tools_str:
                all_tools.extend(t.strip() for t in tools_str.split(';') if t.strip())
        if all_tools:
            print('  Top AI tools:')
            for tool, count in Counter(all_tools).most_common(5):
                print(f'    {tool:<35} {count:>4,}')


analyser = SurveyAnalyser(DATASET_URL, max_rows=500)
analyser.load()
print()
analyser.top_languages(n=8)
print()
analyser.ai_tool_adoption()


---

## Concept Check Questions

> Test your understanding before moving on. Answer each question without referring back to the notebook, then expand to check.

**Q1.** Explain what a **dunder method** is and give two examples with their purpose.

<details><summary>Show answer</summary>

Dunder (double underscore) methods are special methods Python calls implicitly. `__init__(self, ...)` — called when an object is created. `__repr__(self)` — called by `repr()` to produce a string representation. Others: `__len__`, `__add__`, `__iter__`.

</details>

**Q2.** What is the difference between a **class attribute** and an **instance attribute**?

<details><summary>Show answer</summary>

A **class attribute** is defined on the class body and shared by all instances. An **instance attribute** is set on `self` inside `__init__` and unique to each object. Changing a class attribute affects all instances; changing an instance attribute affects only that object.

</details>

**Q3.** Why does a scikit-learn `Pipeline` fit naturally with Python OOP? Name the two core methods they share.

<details><summary>Show answer</summary>

scikit-learn estimators are Python classes. Every estimator implements `fit()` (learn from data) and `transform()` or `predict()` (apply what was learned). A `Pipeline` chains these objects — exactly the `SurveyAnalyser` pattern built in this chapter.

</details>

**Q4.** What is a **generator** and why use one instead of a list?

<details><summary>Show answer</summary>

A generator uses `yield` to produce values one at a time rather than storing the entire sequence in memory. Use a generator when the sequence is large or potentially infinite — it uses O(1) memory instead of O(n).

</details>

**Q5.** Write a decorator `@timer` that prints the execution time of any function.

<details><summary>Show answer</summary>

```python
import time
def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f'{func.__name__} took {time.time()-start:.4f}s')
        return result
    return wrapper
```

</details>



---

## Coding Exercises

> Three exercises per chapter: **🔧 Guided** (fill-in-the-blanks) · **🔨 Applied** (write from scratch) · **🏗️ Extension** (go beyond the chapter)

Exercises use the SO 2025 developer survey dataset.
Expand each **Solution** block only after attempting the exercise.


### Exercise 1 🔧 Guided — SurveyRecord dataclass with validation

Complete the `SurveyRecord` dataclass with a `__post_init__` that validates:
- `years_exp` is between 0 and 50
- `salary` is positive
- `country` is a non-empty string
Raise `ValueError` with a descriptive message for each violation.


In [ ]:
from dataclasses import dataclass

@dataclass
class SurveyRecord:
    respondent_id: int
    years_exp:     float
    salary:        float
    country:       str
    uses_python:   bool = False

    def __post_init__(self) -> None:
        # YOUR CODE HERE
        pass

# Test
try:
    bad = SurveyRecord(1, years_exp=200, salary=50000, country='US')
except ValueError as e:
    print(f'Caught: {e}')
good = SurveyRecord(2, years_exp=5, salary=90000, country='Germany')
print('Valid:', good)

<details><summary>💡 Hint</summary>

Use `if not 0 <= self.years_exp <= 50: raise ValueError(...)` pattern.
`__post_init__` runs automatically after `__init__` in dataclasses.

</details>

<details><summary>✅ Solution</summary>

```python
@dataclass
class SurveyRecord:
    respondent_id: int
    years_exp: float
    salary: float
    country: str
    uses_python: bool = False

    def __post_init__(self):
        if not 0 <= self.years_exp <= 50:
            raise ValueError(f'years_exp must be 0-50, got {self.years_exp}')
        if self.salary <= 0:
            raise ValueError(f'salary must be positive, got {self.salary}')
        if not self.country.strip():
            raise ValueError('country must be a non-empty string')
```

</details>


### Exercise 2 🔨 Applied — Generator pipeline for survey processing

Write a generator pipeline that lazily processes survey records:
1. `read_records(n)` — yields `n` SurveyRecord objects (use random data)
2. `filter_by_country(records, country)` — yields only matching records
3. `add_salary_band(records)` — yields `(record, band)` tuples where
   `band` is `'low'` (<50k), `'mid'` (50k–120k), or `'high'` (>120k)

Chain them together and count how many UK high-earners are in 5,000 records.


In [ ]:
import random
random.seed(42)
COUNTRIES = ['US', 'UK', 'Germany', 'India', 'Canada']

def read_records(n: int):
    # YOUR CODE — yield SurveyRecord objects
    pass

def filter_by_country(records, country: str):
    # YOUR CODE
    pass

def add_salary_band(records):
    # YOUR CODE — yield (record, band) tuples
    pass

uk_high = sum(1 for r, b in add_salary_band(
    filter_by_country(read_records(5000), 'UK'))
    if b == 'high')
print(f'UK high-earners: {uk_high}')

<details><summary>💡 Hint</summary>

Each function should use `yield` — no lists, no `.append()`. This is a lazy pipeline: records are processed one at a time without loading all 5,000 into memory.

</details>

<details><summary>✅ Solution</summary>

```python
def read_records(n):
    for i in range(n):
        yield SurveyRecord(i, random.uniform(0,30),
                           random.gauss(85000,35000), random.choice(COUNTRIES))
def filter_by_country(records, country):
    for r in records:
        if r.country == country: yield r
def add_salary_band(records):
    for r in records:
        if r.salary < 50000: band = 'low'
        elif r.salary < 120000: band = 'mid'
        else: band = 'high'
        yield r, band
```

</details>


### Exercise 3 🏗️ Extension — Context manager for timed ML experiments

Implement an `Experiment` context manager that:
1. Records start time on `__enter__`
2. Records end time and duration on `__exit__`
3. Accepts a `name` parameter
4. Stores all completed experiments in a class-level list `Experiment.history`
5. Implements `Experiment.summary()` that prints a table of all runs

Use it to time three different classifier fits on the salary dataset.


In [ ]:
import time
from typing import Optional

class Experiment:
    history: list = []

    def __init__(self, name: str) -> None:
        self.name = name
        self._start: Optional[float] = None

    def __enter__(self) -> 'Experiment':
        # YOUR CODE
        return self

    def __exit__(self, *args) -> None:
        # YOUR CODE
        pass

    @classmethod
    def summary(cls) -> None:
        # YOUR CODE
        pass

# Example usage:
# with Experiment('GBM depth=3') as exp:
#     clf.fit(X_tr, y_tr)

<details><summary>💡 Hint</summary>

In `__enter__`: `self._start = time.perf_counter()`. In `__exit__`: compute duration and append `{'name': self.name, 'duration_s': ...}` to `Experiment.history`.

</details>

<details><summary>✅ Solution</summary>

```python
class Experiment:
    history = []
    def __init__(self, name): self.name = name; self._start = None
    def __enter__(self):
        self._start = time.perf_counter(); return self
    def __exit__(self, *args):
        duration = time.perf_counter() - self._start
        Experiment.history.append({'name': self.name, 'duration_s': round(duration, 4)})
    @classmethod
    def summary(cls):
        print(f'{'Experiment':<30} {'Time (s)':>10}')
        for e in cls.history:
            print(f'{e["name"]:<30} {e["duration_s"]:>10.4f}')
```

</details>


---

## Chapter 2 Summary

Chapter 2 covers the techniques that separate beginner Python from professional Python.
Every pattern here reappears in the data science and ML chapters ahead.

### Key Takeaways

- **Modules:** `import module as alias` is the universal data science convention.
  `collections.Counter` and `collections.defaultdict` are indispensable.
- **File I/O:** Always use `with open(...)` -- it guarantees the file closes on error.
  `csv.DictReader` and `json.load()` are your primary tools for raw file parsing.
- **Error handling:** `try/except/else/finally` -- use specific exception types,
  never use bare `except:`, and always re-raise or log unexpected errors.
- **OOP:** A class bundles data (attributes) and behaviour (methods).
  `__init__` stores config; `fit()`-style methods learn from data; attributes
  set by fit end in `_` by convention.
- **The scikit-learn bridge:** `StandardScaler`, `Pipeline`, `RandomForestClassifier` --
  all of them are just classes with `__init__`, `fit()`, and `transform()`/`predict()`.
  You now understand exactly how they work.
- **Comprehensions** are faster and more readable than equivalent for-loops.
- **Generators** use `yield` and produce values on demand -- essential for large data.
- **Decorators** wrap functions to add behaviour (timing, validation, logging)
  without modifying the original function.

### Project Thread Status

| Task | Status |
|------|--------|
| Built `SurveyLoader` class with `load()`, `validate()`, `get_records()` | Done |
| Error handling for network failures and malformed rows | Done |
| Built `SurveyAnalyser` subclass with language and AI tool analytics | Done |
| Followed scikit-learn fit/transform pattern throughout | Done |

---

### What's Next: Chapter 3 -- NumPy and Pandas

Chapter 3 introduces the data science stack. With NumPy and Pandas,
everything from Chapter 2 that we did manually -- loading, cleaning,
summarising -- becomes 10-100x faster and dramatically more concise.
We load the full 15,000-row SO 2025 dataset and build a complete cleaning pipeline.

---

*End of Chapter 2 -- Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
